In [1]:
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"

## Hash and Equality

The `hash()` built-in function works directly with built-in types and falls back to the `__hash__` method for user-defined classes. 

> When correctly implemented, hashing guarantees that different hash codes always imply different objects, but the **reverse is not true**: different objects don’t always have different hash codes. When different objects have the same hash code, that’s a hash collision.

To be effective hash table indices, hash values should be evenly distributed across the range of integers. This is known as the uniform hashing assumption:

> The hash functions that uniformly and independently distribute keys among the integer values between 0 and $M–1$. And $M$ is the size of the hash table (chosen to be prime). (Algorithms Sedgewick and Wayne)

Universal hashing has a certain mathematical property that guarantees a low number of collisions in expectation.

### Integers

In [2]:
# Integer
hash(12391)

# Unique case: does not hash to itself
hash(-1)

hash(-3)

# Long
hash(12345678910111213141516)

12391

-2

-3

195438781095727862

### Floats

In [3]:
hash(12.342)

hash(1.232322412312341232)

788598309151084556

535699010314073601

### Strings, Bytes, and DateTimes

Starting with Python 3.3, a random salt value is included when computing hash codes for `str`, `bytes`, and `datetime` objects, which is constant within a Python process but varies between interpreter runs.

```python
from datetime import datetime
import os

def hash_it(x):
    # Get the current time to log when the hash is calculated
    current_time = datetime.now().strftime("%H:%M:%S.%f")
    # Retrieve the PYTHONHASHSEED environment variable if set
    hash_seed = os.environ.get('PYTHONHASHSEED', 'not set')
    # Return the hash along with the current time and seed information
    return (hash(x), current_time, hash_seed)

def main():
    now_datetime = datetime.now()
    string = "Yang Wu"
    bytes_string = b"Yang Wu"
    
    for item in [now_datetime, string, bytes_string]:
        print(f"Hash for the same item {item} multiple times:")
        print({key: hash_it(item) for key in range(3)})
        
    return 0

if __name__ == '__main__':
    
    main()
```

In [4]:
!python3 ../scripts/random_salt.py

Hash for the same item 2024-05-12 18:08:04.580142 multiple times:
{0: (-2640074976138239007, '18:08:04.580334', 'not set'), 1: (-2640074976138239007, '18:08:04.580428', 'not set'), 2: (-2640074976138239007, '18:08:04.580452', 'not set')}
Hash for the same item Yang Wu multiple times:
{0: (-3333417715881684127, '18:08:04.580524', 'not set'), 1: (-3333417715881684127, '18:08:04.580549', 'not set'), 2: (-3333417715881684127, '18:08:04.580566', 'not set')}
Hash for the same item b'Yang Wu' multiple times:
{0: (-3333417715881684127, '18:08:04.580617', 'not set'), 1: (-3333417715881684127, '18:08:04.580637', 'not set'), 2: (-3333417715881684127, '18:08:04.580656', 'not set')}


In [5]:
!python3 ../scripts/random_salt.py

Hash for the same item 2024-05-12 18:08:04.795107 multiple times:
{0: (-4665693135182612666, '18:08:04.795189', 'not set'), 1: (-4665693135182612666, '18:08:04.795296', 'not set'), 2: (-4665693135182612666, '18:08:04.795323', 'not set')}
Hash for the same item Yang Wu multiple times:
{0: (-8504513589875496229, '18:08:04.795453', 'not set'), 1: (-8504513589875496229, '18:08:04.795504', 'not set'), 2: (-8504513589875496229, '18:08:04.795528', 'not set')}
Hash for the same item b'Yang Wu' multiple times:
{0: (-8504513589875496229, '18:08:04.795651', 'not set'), 1: (-8504513589875496229, '18:08:04.795706', 'not set'), 2: (-8504513589875496229, '18:08:04.795734', 'not set')}


Each time we run the script above, the hash values for the same `str`, `bytes`, and `datetime` objects will be different. But the hash values without each run will be the same.

## Set

<center>
<img src="../images/set_flowchart.png" width="690" height="380">
</center>


### Step 0: Initialize the Hash Table

The hash table for a `set` starts with 8 empty buckets. As elements are added, Python makes sure at least $\frac{1}{3}$ of the buckets are empty, but resizes the hash table by doubling or quadrupling when more space is needed.

### Step 1: Compute the Hash Code of the Element

In [6]:
days = ["Mon", "Tue", "Wed", "Thu", "Fri"]

for day in days:
    print(hash(day))

-517213032146247581
-6862824312894268680
2067417196853314201
-9051317402501595766
-4608781622546238704


**Note: From this step forward we will use a fixed set of hash codes for `Mon`, `Tue`, `Wed`, `Thu`, and `Fri` to illustrate the probing process. But in reality, the hash codes are different each time a Python process is started.** 

The hash codes for the days of the week are:

In [7]:
fixed_hash_codes = {
    "Mon": 4199492796428269555,
    "Tue": 2414279730484651250,
    "Wed": -5145319347887138165,
    "Thu": -1139383146578602409,
    "Fri": 7021641685991143771,
}

### Step 2: Probe Hash Table at Index Derived from Hash Code

Python takes the modulus of the hash code with the table size to find a hash table index:

In [8]:
for day in fixed_hash_codes:
    print(f"Index for {day} is {fixed_hash_codes[day] % 8}")

Index for Mon is 3
Index for Tue is 2
Index for Wed is 3
Index for Thu is 7
Index for Fri is 3


Probing consists of computing the index from the hash, then looking at the corresponding bucket in the hash table.

### Step 3: Place the Element in the Empty Bucket

#### Monday

In [9]:
print("The hash code for 'Mon' is", fixed_hash_codes["Mon"])

print("The offset is", fixed_hash_codes["Mon"] % 8)

The hash code for 'Mon' is 4199492796428269555
The offset is 3


Python stores the hash code of the new element, `4199492796428269555`, in the hash code field at offset `3`, and a pointer to the string object `Mon` in the element field.

<center>
<img src="../images/monday_set.png" width="480" height="380">
</center>

#### Tuesday

In [10]:
print("The hash code for 'Tue' is", fixed_hash_codes["Tue"])

print("The offset is", fixed_hash_codes["Tue"] % 8)

The hash code for 'Tue' is 2414279730484651250
The offset is 2


The hash code and the pointer to element `Tue` are placed in the bucket at offset `2`, which is also empty:

<center>
<img src="../images/tuesday_set.png" width="480" height="380">
</center>

Notice the order insert is not reflected in the order of the elements in the hash table.

#### Wednesday

In [11]:
print("The hash code for 'Wed' is", fixed_hash_codes["Wed"])

print("The offset is", fixed_hash_codes["Wed"] % 8)

The hash code for 'Wed' is -5145319347887138165
The offset is 3


When adding `Wed` to the set, Python 

1. Computes the hash `-5145319347887138165` and derives the index `3` from `hash % 8`

2. Probes bucket `3` and sees that it is already taken but the hash code stored there, `4199492796428269555` is different

3. Probes the next bucket and finds it empty, so `Wed` ends up at index `4`

This is called an index collision.

<center>
<img src="../images/wednesday_set.png" width="480" height="380">
</center>

#### Thursday and Friday

In [12]:
print("The hash code for 'Thu' is", fixed_hash_codes["Thu"])

print("The offset is", fixed_hash_codes["Thu"] % 8)

print("The hash code for 'Fri' is", fixed_hash_codes["Fri"])

print("The offset is", fixed_hash_codes["Fri"] % 8)

The hash code for 'Thu' is -1139383146578602409
The offset is 7
The hash code for 'Fri' is 7021641685991143771
The offset is 3


Adding the next element, `Thu` results in no collision, and it lands in its natural bucket, at index `7`.

For `Fri`, Python:

1. Probes the hash table starting at index `3`, which is taken by `Mon`, and sees that the hash codes for `Fri` and `Mon` don't match (Index Collision)

2. Probes the next bucket at index `4`, which is taken by `Wed`, and sees that the hash codes for `Fri` and `Wed` don't match (Index Collision)

3. Probes the next bucket at index `5`, which has `-1` in the hash code field, marking an empty bucket, so `Fri` is placed there

<center>
<img src="../images/thursday_friday_set.png" width="480" height="380">
</center>

### Resizing the Hash Table

The hash table for the set `{Mon, Tue, Wed, Thu, Fri}` has 8 buckets, but only 5 are used ($\frac{5}{8} = 0.625$). When the capacity exceeds $\frac{2}{3} \approx 0.667$, Python resizes the hash table. Based
on the following [line](https://github.com/python/cpython/blob/main/Objects/setobject.c#L199C1-L199C75) in the CPython source code, if the set needs resizing, the resizing behavior depends on the current size of the set:

* If the used count is greater than 50,000, the set size is doubled (`so->used*2`).

* If the used count is 50,000 or less, the set size is quadrupled (`so->used*4`).

### Hash Collision

It is possible that **two different objects** have the **same hash code**. This is known as a **hash collision**.

This explains why hashable objects must implement both `__hash__` and `__eq__`.

#### Insertion 

If the hash code of the new element to be inserted matches the hash code of an existing element, Python compares the new element with the existing element using the `__eq__` method:

* If the element values are equal, Python does not add the new element to the set. 

* If the elements values are not equal, Python continues probing the hash table until it finds an empty bucket.

#### Lookup

When a lookup is requested `element in set` and the hash code of `element` collides with the hash codes of other elements, Python again needs to compare the element's value using the `__eq__` method:

* If the equality check returns true, the element is found

* If the equality check returns false, Python continues probing the hash table until it finds the element or exhausts all buckets, which means the element is not in the set.

Evidently, hash collision results in additional probing for both insertion and lookup operations, degrading the performance of the hash table.

### Practical Consequences

1. Elements must implement `__hash__` and `__eq__` methods.

2. The bucket for an element can be located directly by computing the hash code of the element and deriving an index offset, with the possible overhead of a small number of probes to find a matching element or an empty bucket.

3. The most compact internal data structure for a container would be an array of pointers. Hash tables use more memory due to additional hash codes and maintaining at least $\frac{1}{3}$ of the buckets empty to minimize collisions.

4. The order of elements in a set is influenced by the insertion sequence but is neither reliable nor predictable.

5. Adding elements may cause a rehash to maintain bucket availability, altering the order of existing elements so different index collisions may occur.

---

## Linear Probing

Python uses linear probing to resolve collisions. This is used for both index collisions and hash collisions for both sets and dictionaries.

> Linear probing is a simple collision resolution strategy that consists of creating **new** indices to probe in the hash table until an either an empty bucket is found or data to be searched is found.

We need a better strategy than simply looking at the next bucket since that would lead to clusters of occupied buckets. That is, if each time we find a collision, we simply probe the next bucket, we would end up with a lot of contiguous filled buckets. Then, the next time we need to probe, we would have to go through all of them to find the next empty bucket or the data we are looking for.

## Psuedo code for A Modified Linear Probing

```python
def index_sequence(key, mask=0b111, PERTURB_SHIFT=5): 
    perturb = hash(key)
    i = perturb & mask
    yield i
    while True:
        perturb >>= PERTURB_SHIFT
        i = (i * 5 + perturb + 1) & mask yield i
```

#### Function Definition

- `def index_sequence(key, mask=0b111, PERTURB_SHIFT=5)`: 
  - `key`: The input for which the hash index is calculated.
  - `mask=0b111`: Default value is 7 (in binary, `0b111`). This is used to limit the range of the indices. This can be set to a different value to control the number of buckets.
  - `PERTURB_SHIFT=5`: Default shift value used in the perturbation process. This default comes from the properties of a linear congruential generator (LCG), which is used in generating random numbers.

#### Initialization

- `perturb = hash(key)`: Computes the hash of the `key`, storing it in `perturb`.
- `i = perturb & mask`: Applies a bitwise AND between `perturb` and `mask` to get the initial index. This operation ensures that the index stays within the bounds of the hash table defined by `mask`.

#### Yield the First Index

- `yield i`: The first computed index is yielded back to the caller. This is the initial index in the hash table to be probed.

#### Infinite Loop for Index Generation

- `while True`: Starts an infinite loop to generate further indices if needed.
  - `perturb >>= PERTURB_SHIFT`: Right-shifts the `perturb` value by `PERTURB_SHIFT` positions. This operation gradually reduces `perturb`, altering its value for subsequent index calculations.
  - `i = (i * 5 + perturb + 1) & mask`: Calculates the next index using the previous index `i`, modified `perturb`, and an additive constant `1`. The entire result is again constrained by `mask`.
  - `yield i`: Yields the next index in the sequence back to the caller.

This function can generate a sequence of indices for placing or finding an item in a hash table, particularly useful when handling collisions using open addressing. The use of both perturbation and the `mask` helps in distributing the indices more uniformly and ensuring they stay within a certain range.

---

## Dict 

Consider the following dictionary:

In [13]:
swimmers = {"Mon": 14, "Tue": 12, "Wed": 14, "Thu": 11}

swimmers

{'Mon': 14, 'Tue': 12, 'Wed': 14, 'Thu': 11}

### Old Dict Hash Table

Prior to the introduction of compact Dict in Python 3.6, the hash table looks as follows:

<center>
<img src="../images/old_dict_hash_table.png" width="540" height="380">
</center>

In a 64-bit Python, each bucket holds three 64-bit fields:

1. The hash code of the key
2. A pointer to the key object
3. A pointer to the value object

That is 24 bytes or 192 (1 byte = 8 bit) bits per bucket. Each bucket is a struct with the hash code of the key, a pointer to the key, and a pointer to the value.

### Compact Dict

As proposed by Raymon Hettinger [here](https://mail.python.org/pipermail/python-dev/2012-December/123028.html), an optimization can be made such that

1. The hash code and pointers to key and value were held in an `entries` array with no empty rows
2. The actual hash table were a sparse array with much smaller buckets, holding indexes into the `entries` array

<center>
<img src="../images/compact_dict_hash_table.png" width="640" height="280">
</center>

In the diagram above:

* Hash codes and pointers to keys and values are stored in **insertion order** in the `entries` array
* The entry offsets derived from the hash codes are held in the `indices` sparse array with negative values for special purposes, e.g. -1 for empty and -2 for deleted

Assuming a 64-bit build of CPython, the 4-item `swimmers` dictionary would take:

* 192 bytes of memory in the old scheme: 24 bytes per bucket, times 8 rows 
* 104 bytes of memory using the compact scheme: 96 bytes in `entries` (`24 * 4`), plus 8 bytes for the buckets in `indices` configured as an array of `8` bytes (the default `indices` table is initially set up with 8 buckets)

### Step 0: Set up `indices` 

* The `indices` table is initially set up as an array of signed bytes, with 8 buckets, each initialized with `-1` to signal "empty bucket". Up to 5 of these buckets will eventually hold indices to rows in the `entries` array, leaving $\frac{1}{3}$ of the buckets empty before resizing.

* The `entries` array hods hash code and pointers for keys and values in **insertion order*.

In [14]:
indices = [-1] * 8

indices

[-1, -1, -1, -1, -1, -1, -1, -1]

### Step 1: Compute the Hash Code of the Key

To add the key-value pair `('Mon', 14)` to the swimmers dictionary, Python first calls `hash('Mon')` to compute the hash code of that key.

### Step 2: Probe `entries` using `indices`

Python computes `hash('Mon') % len(indices)`. Offset `3` in indices holds `-1`: it’s an empty bucket:

In [15]:
print(f"The derived index for 'Mon' is {fixed_hash_codes['Mon'] % len(indices)}")

The derived index for 'Mon' is 3


### Step 3: Place the Key-Value Pair in the `entries` Array, Update `indices`

#### Monday

The `entries` array is empty, so the next available offset is `0`. Python

1. Places the index `0` at offset `3` in `indices`
2. Stores the hash code of `Mon`, the pointer to the key `'Mon'`, and the pointer to the value `14` in the `entries` array at offset `0` in the `entries` array

<center>
<img src="../images/monday_dict.png" width="660" height="300">
</center>

So `indices[3]` holds the offset of the first entry `entries[0]`:

In [16]:
indices[fixed_hash_codes["Mon"] % len(indices)] = 0

indices

[-1, -1, -1, 0, -1, -1, -1, -1]

#### Tuesday

To add `(Tue, 12)` to the dictionary, Python

1. Computes the hash code of `Tue`

2. Compute the offset into `indices`, e.g. `hash('Tue') % len(indices) = 2`

3. Checks that `indices[2]` is empty with value `-1`, so no collision

4. Places the next available `entries` offset `1` at `indices[2]`

5. Stores the hash code of `Tue`, the pointer to the key `'Tue'`, and the pointer to the value `12` in the `entries` array at offset `1`

<center>
<img src="../images/tuesday_dict.png" width="660" height="300">
</center>

So `indices[2]` holds the offset of the second entry `entries[1]`:

In [17]:
indices[fixed_hash_codes["Tue"] % len(indices)] = 1

indices

[-1, -1, 1, 0, -1, -1, -1, -1]

#### Wednesday

Adding `(Wed, 14)` to the dictionary, Python

1. Computes the hash code of `Wed`

2. Compute the offset into `indices`, e.g. `hash('Wed') % len(indices) = 3`

3. Checks that `indices[3]`, which has `0`, pointing to an existing entry, so there is an index collision

   1. Compares the hash code of `Wed` with the hash code of `Mon` at `entries[0]`, which don't match

   2. Probes the next bucket in `indices` at offset `4`, which is empty with value `-1`

4. Places the next available `entries` offset `2` at `indices[4]`

5. Stores the hash code of `Wed`, the pointer to the key `'Wed'`, and the pointer to the value `14` in the `entries` array at offset `2`

<center>
<img src="../images/wednesday_dict.png" width="660" height="300">
</center>

So `indices[4]` holds the offset of the third entry `entries[2]`:

In [18]:
indices[4] = 2

indices

[-1, -1, 1, 0, 2, -1, -1, -1]

### Resizing

Based on this block of [comments](https://github.com/python/cpython/blob/main/Objects/dictobject.c#L603C1-L612C4):

1. Up to Python version 3.2, the growth rate was set to `used * 4`, which means the dictionary size was quadrupled upon reaching maximum load

2. In Python version 3.3.0, the growth rate was reduced to `used * 2`, effectively doubling the dictionary size

3. From Python versions 3.4.0 to 3.6.0, the growth rate was adjusted to `used * 2 + capacity / 2`. This formula implies that the size increase was dependent not only on the number of used entries but also included half of the current capacity, allowing for a more gradual increase in size.

4. Currently, i.e. Python $3.11$, the dictionary size increases by $\times 3$ once the load threshold is exceeded.

**Initial Conditions and Growth Trigger:**

- The smallest size allocated for a dictionary is 8 elements.

- Resizing occurs when the dictionary is more than two-thirds full.

**Example of Growth Progression:**

<div align="center">

| Insertion Event | Dictionary Size Before Insert | Dictionary Size After Insert |
|-----------------|-------------------------------|------------------------------|
| 6th item        | 8                             | 18                           |
| 13th item       | 18                            | 39                           |
| ...             | 39                            | 81                           |
| ...             | 81                            | 165                          |
| ...             | 165                           | 333                          |
| ...             | 333                           | 669                          |
| ...             | 669                           | 1,341                        |
| ...             | 1,341                         | 2,685                        |
| ...             | 2,685                         | 5,373                        |
| ...             | 5,373                         | 10,749                       |
| ...             | 10,749                        | 21,501                       |
| ...             | 21,501                        | 43,005                       |
| And so on       | ...                           | ...                          |

</div>

### Hash Collision

Obviously, hash collision is not ideal since the additional probing degrades the performance of the dictionary for both insertion and lookup operations.

Python dictionaries store both the hash code and the pointer to the actual key object, so even if multiple keys have the same hash, Python can distinguish between them by directly comparing the keys.

#### Insertion

When adding a key-value pair to a dictionary and a hash collision occurs:

* Python will derive the index from the hash code for the new key and probe the `indices` array to find that there is an existing offset at that index (i.e. not `-1`).

* Python will compare the actual key objects to determine if the key to be inserted and the existing key are the same. If the keys are the same, there is no insertion to do since the key-value pair is already in the dictionary. The key will be updated with the new value.

* If the keys don't match, Python will continue probing the sparse `indices` array until it finds an empty bucket `i` marked by `-1`; it will insert the next available offset into `entries` at `indices[i]` and store the hash code and pointers to the key and value in the `entries` array at `offset`.

#### Lookup

If the keys `Mon` and `Wed` have a hash collision and a lookup for `Wed` is requested, simply computing the index from the hash code will not suffice to disambiguate the keys.

Python compares the requested key object to the existing key object whose hash codes collided to find the correct key-value pair for `Wed`. This additional probing is necessary to handle hash collisions, but it degrades the performance of the dictionary.

### Key Sharing Dictionary

Instances of user-defined classes hold their attributes in a `__dict__` attribute, which is a regular dictionary. An instance `__dict__` maps attribute names to attribute values. Most of the time, all instances have the same attributes with different values.

One [optimization](https://peps.python.org/pep-0412/) proposed by Mark Shannon:

1. Store each attribute hash code and pointer only once and link it to the class
2. Store attribute values in parallel arrays of pointers attached to each instance

This optimization is known as a **key-sharing dictionary**. 

Below is an example of split storage for the `__dict__` of a class and three instances of that class:

<center>
<img src="../images/key_sharing_dict.png" width="660" height="300">
</center>

### Practical Consequences

1. Keys must implement `__hash__` and `__eq__` methods. But values do not have to be hashable.

2. Item ordering is preserved in the `entries` table, which is an implementation detail since 3.7.

3. For key sharing dictionary:

   - **Default dictionary layout**: combined-table is still used when creating dictionaries using `dict()`.

   - **Split-table dictionary**: This is used for the `__dict__` of a class's first instance. The keys table is then cached in the class after the creation of the first instance.

     - Subsequent instances only store their values, sharing the `keys` table.

     - If an instance gets a new attribute not found in the shared keys table, then this instance’s `__dict__` is converted back to combined-table form.
  
     - If the instance is the only one in its class, `__dict__` reverts to split-table for potential future instance attribute sharing even if a new attribute is added.

---

## Finding the Mask 

The mask is used to limit the range of the indices. To find the mask for a dictionary with an arbitrary number of elements, $N$:

1. Find the minimum number of buckets that the dictionary must have to still be $\frac{2}{3}$ full:

    \begin{align*}
    N \times (\frac{2}{3} + 1)
    \end{align*}

2. Find the smallest dictionary size that will hold this number of elements: (8; 18; 39; 81; 165; 333; 669; 1,341; 2,685;)

3. Find the number of bits necessary to hold this number using `bin(dict_size - 1)`

Example:

In [19]:
n = 1039

min_buckets = n * (2 / 3 + 1)

print(f"The minimum number of buckets for {n} elements is {min_buckets}")

The minimum number of buckets for 1039 elements is 1731.6666666666665


In [20]:
dict_size = 2685

bin(dict_size - 1)

'0b101001111100'

In [21]:
# Derive the offset for 'Yang Wu'
hash("Yang Wu") & 0b101001111100

4